In [115]:
#import coordinates
df_geo_coordinates = pd.read_csv("C:\\Users\k9117\Desktop\Geospatial_Coordinates.csv")
df_geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [116]:
#merge dataframes
df_torontodata = pd.merge(canada_df, df_geo_coordinates, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
# remove the "Postal Code" column
df_torontodata.drop("Postal Code", axis=1, inplace=True)
df_torontodata.head()


,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


GEO DATA and Segmentation


In [117]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [118]:
from geopy.geocoders import Nominatim

In [119]:
from pandas.io.json import json_normalize

In [120]:
#funding Toronto coordinates
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto city are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto city are 43.6534817, -79.3839347.


In [121]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [122]:
import folium


In [124]:
# create map of Toronto coordinates extracted above
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [125]:
# adding markers to the map
for lat, lng, borough, neighborhood in zip(
        df_torontodata['Latitude'], 
        df_torontodata['Longitude'], 
        df_torontodata['Borough'], 
        df_torontodata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [39]:
# "denc" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto
df_torontodata_denc = df_torontodata[df_torontodata['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_torontodata_denc.head()

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [40]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_torontodata_denc['Latitude'], 
        df_torontodata_denc['Longitude'], 
        df_torontodata_denc['Borough'], 
        df_torontodata_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc

In [126]:
# printing 3rd neighborhood name (random;y selected)
neighborhood_name = df_torontodata_denc.loc[2, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'India Bazaar, The Beaches West'.


In [128]:
#finding coordiates of selected neighborhood
neighborhood_latitude = df_torontodata_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_torontodata_denc.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of India Bazaar, The Beaches West are 43.67635739999999, -79.2930312.


In [1]:
# Foursq. cred - (output is hided)
CLIENT_ID =''
CLIENT_SECRET =''
VERSION = '20180604'
print ('your credentials:')
print ('CLIENT_ID:'+CLIENT_ID)
print ('CLIENT_SECRET:' + CLIENT_SECRET)


your credentials:
CLIENT_ID:
CLIENT_SECRET:


In [131]:
# Using Foursq data finding top 30 venues within 500 m. radius
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [132]:
#Extracting category of venues
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [133]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 

# columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

E:\python\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
